In [1]:
# For inline plotting
%matplotlib inline

# For auto reloading
%load_ext autoreload
%autoreload 2


In [2]:
import pickle
from pathlib import Path

from mab import GleapAgent, GleapEpsilonGreedyAgent, UcbAgent, UniformlyRandomAgent
import simulation_config


# Load all the simulation objects
# date = "20210820_211857"
# date = "20220127_143645"
date = "20220127_151410"
# date = "20220128_000548"
# date = "20220219_215450"
# date = "20220221_085656"
simulation_list = []
pathlist = Path("./simulations_data/ht/{:s}".format(date)).glob('*.pkl')
for path in pathlist:
    with open(path, "rb") as f:
        simulation_list.append(pickle.load(f))


# Sort the simulation list in the ascending order of num_round
def value_func(simulation_index):
    simulation = simulation_list[simulation_index]
    
    value = simulation.num_round
    
    return value


# A dictionary. Key: h_function_index. Value: a list of simulation index, sorted in the ascending order of num_round.
h_function_index_simulation_index_list_dict = {}

# A dictionary. Key: num_round. Value: 1.
# This dictionary is used as a set of num_round.
num_round_dict = {}

for simulation_index, simulation in enumerate(simulation_list):
    h_function_index = simulation.h_function_index
    num_round = simulation.num_round
    
    if h_function_index in h_function_index_simulation_index_list_dict:
        h_function_index_simulation_index_list_dict[h_function_index].append(simulation_index)
    else:
        h_function_index_simulation_index_list_dict[h_function_index] = [ simulation_index ]
    
    if num_round not in num_round_dict:
        num_round_dict[num_round] = 1
h_function_index_list = sorted([ h_function_index for h_function_index in h_function_index_simulation_index_list_dict ])
num_round_list = sorted([ num_round for num_round in num_round_dict ])
for h_function_index in h_function_index_simulation_index_list_dict:
    sorted_list = sorted(h_function_index_simulation_index_list_dict[h_function_index], key=lambda i: value_func(i))
    h_function_index_simulation_index_list_dict[h_function_index] = sorted_list
    for simulation_index in sorted_list:
        simulation = simulation_list[simulation_index]


for h_function_index in h_function_index_list:
    print("h_function_index = {:d}".format(h_function_index))
    simulation_index_list = h_function_index_simulation_index_list_dict[h_function_index]
    for simulation_index in simulation_index_list:
        simulation = simulation_list[simulation_index]
        print(simulation.agent.name)
    print()


print("num_round_list:")
for num_round in num_round_list:
    print("{:d}".format(num_round))


h_function_index = 0
G-LEAP, $H = 0$, $T = 200$
G-LEAP, $H = 0$, $T = 400$
G-LEAP, $H = 0$, $T = 600$
G-LEAP, $H = 0$, $T = 800$
G-LEAP, $H = 0$, $T = 1000$

h_function_index = 1
G-LEAP, $H = 20$, $T = 200$
G-LEAP, $H = 40$, $T = 400$
G-LEAP, $H = 60$, $T = 600$
G-LEAP, $H = 80$, $T = 800$
G-LEAP, $H = 100$, $T = 1000$

h_function_index = 2
G-LEAP, $H = 200$, $T = 200$
G-LEAP, $H = 400$, $T = 400$
G-LEAP, $H = 600$, $T = 600$
G-LEAP, $H = 800$, $T = 800$
G-LEAP, $H = 1000$, $T = 1000$

h_function_index = 3
G-LEAP, $H = 1060$, $T = 200$
G-LEAP, $H = 2397$, $T = 400$
G-LEAP, $H = 3838$, $T = 600$
G-LEAP, $H = 5348$, $T = 800$
G-LEAP, $H = 6908$, $T = 1000$

num_round_list:
200
400
600
800
1000


In [3]:
import math
from pathlib import Path
import shutil

import numpy as np
import matplotlib
print("matplotlib.__version__:", matplotlib.__version__)
print('Default backend: ' + matplotlib.get_backend())
matplotlib.use("module://mplcairo.base")
print('Backend is now ' + matplotlib.get_backend())
import matplotlib.font_manager as fm
import matplotlib.pyplot as plt
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42

from mab import GleapAgent, GleapEpsilonGreedyAgent


def calculate_metrics(simulation_index):
    simulation = simulation_list[simulation_index]
    
    # For reward
    time_averaged_compound_reward_array = np.mean(simulation.time_averaged_compound_reward_2darray, axis=0)
    aggregated_compound_reward_array = np.mean(simulation.aggregated_compound_reward_2darray, axis=0)
    
    # For regret
    time_averaged_regret_array = np.mean(simulation.time_averaged_regret_2darray, axis=0)
    cumulative_regret_array = np.mean(simulation.cumulative_regret_2darray, axis=0)
    
    # For time-averaged total prediction accuracy
    time_averaged_total_prediction_accuracy_array = np.mean(simulation.time_averaged_total_prediction_accuracy_2darray, axis=0)
    
    # For queue length
    if isinstance(simulation.agent, (GleapAgent, GleapEpsilonGreedyAgent)):
        q_array = np.mean(simulation.q_2darray, axis=0)
    else:
        q_array = None
    
    # For time-averaged total prediction latency
    time_averaged_total_prediction_latency_array = np.mean(simulation.time_averaged_total_prediction_latency_2darray, axis=0)
    
    # For time-averaged energy consumption
    time_averaged_energy_consumption_array = np.mean(simulation.time_averaged_energy_consumption_2darray, axis=0)
    
    return {\
            "time_averaged_compound_reward_array": time_averaged_compound_reward_array,\
            "aggregated_compound_reward_array": aggregated_compound_reward_array,\
            "time_averaged_regret_array": time_averaged_regret_array,\
            "cumulative_regret_array": cumulative_regret_array,\
            "time_averaged_total_prediction_accuracy_array": time_averaged_total_prediction_accuracy_array,\
            "q_array": q_array,\
            "time_averaged_total_prediction_latency_array": time_averaged_total_prediction_latency_array,\
            "time_averaged_energy_consumption_array": time_averaged_energy_consumption_array\
           }


def plot_ht_simulations():
    # mkdir
    figure_path = Path("./simulations_data/ht/{:s}/figures".format(date))
    figure_path.mkdir(parents=True, exist_ok=True)
    shutil.rmtree(figure_path)
    figure_path.mkdir(parents=True, exist_ok=True)
    chinese_figure_path = Path("./simulations_data/ht/{:s}/chinese_figures".format(date))
    chinese_figure_path.mkdir(parents=True, exist_ok=True)
    shutil.rmtree(chinese_figure_path)
    chinese_figure_path.mkdir(parents=True, exist_ok=True)
    
    
    color_list = [ '#D55E00', '#0072B2', '#009E73', '#E69F00', '#CC79A7' ]
    marker_list = [ 'D', 's', 'o', 'v', '*' ]
    MARKER_SIZE = 25.0
    MARKER_SIZE_CHINESE = 14.0
    LINE_WIDTH = 6.0
    LINE_WIDTH_CHINESE = 3.0
    FONT_SIZE = 36.0
    FONT_SIZE_CHINESE = 32.0
    FONT_SIZE_TICK = 32.0
    FONT_SIZE_TICK_CHINESE = 26.0
    FONT_SIZE_TICK_OFFSET_TEXT = 28.0
    FONT_SIZE_TICK_OFFSET_TEXT_CHINESE = 24.0
    FIG_SIZE = (16,9)
    FIG_SIZE_CHINESE = (8,6)
    DPI = 120
    DPI_CHINESE = 300
    GRID = True
    GRID_LINE_STYLE = (0,(8,4))
    FONT_WEIGHT = "normal"
    BAR_EDGE_COLOR = "#333333"
    CHINESE_FONT_WEIGHT = "normal"
    if CHINESE_FONT_WEIGHT == "bold":
        CHINESE_FP = fm.FontProperties(fname='./font/source_han_serif_sc_bold.otf', math_fontfamily="stix")
    else:
        CHINESE_FP = fm.FontProperties(fname='./font/source_han_serif_sc_regular.otf', math_fontfamily="stix")
    SAVE_CHINESE_PDF = True
    
    
    converged_time_averaged_compound_reward_array_list = []
    converged_time_averaged_regret_array_list = []
    for h_function_index in h_function_index_list:
        simulation_index_list = h_function_index_simulation_index_list_dict[h_function_index]
        
        length = len(simulation_index_list)
        converged_time_averaged_compound_reward_array = np.zeros(length, dtype=float)
        converged_time_averaged_regret_array = np.zeros(length, dtype=float)
        
        for index, simulation_index in enumerate(simulation_index_list):
            metrics = calculate_metrics(simulation_index)
            
            converged_time_averaged_compound_reward_array[index] = metrics["time_averaged_compound_reward_array"][-1]
            converged_time_averaged_regret_array[index] = metrics["time_averaged_regret_array"][-1]
        
        converged_time_averaged_compound_reward_array_list.append(converged_time_averaged_compound_reward_array)
        converged_time_averaged_regret_array_list.append(converged_time_averaged_regret_array)
    
    
    ##########################################################################################
    # Beginning for the converged time-averaged compound reward
    
    fig, ax = plt.subplots(figsize=FIG_SIZE, dpi=DPI)
    
    ax.set_ylim(0.0, 0.8)
    
    label_list = [ "$H_{\mathrm{min}} = 0$", "$H_{\mathrm{min}} = 0.1 T$", "$H_{\mathrm{min}} = T$", "$H_{\mathrm{min}} = T \log T$" ]

    x_array = np.arange(len(num_round_list))
    width = 0.18 # The width of the bars
    
    for index, converged_time_averaged_compound_reward_array in enumerate(converged_time_averaged_compound_reward_array_list):
        ax.bar(x_array + (index - 1.5) * width, converged_time_averaged_compound_reward_array, width, color=color_list[index], label=label_list[index], edgecolor=BAR_EDGE_COLOR, zorder=3)

    
    legend = ax.legend(shadow=True, prop=fm.FontProperties(size=32.0, style="normal"))
    plt.grid(GRID, linestyle=GRID_LINE_STYLE, zorder=0)
    plt.ticklabel_format(style='sci', axis='y', scilimits=(0, 0), useMathText=True)
    plt.setp(ax.get_xticklabels(), fontsize=FONT_SIZE_TICK, fontweight=FONT_WEIGHT)
    plt.setp(ax.get_yticklabels(), fontsize=FONT_SIZE_TICK, fontweight=FONT_WEIGHT)
    plt.setp(ax.xaxis.get_offset_text(), fontsize=FONT_SIZE_TICK_OFFSET_TEXT, fontweight=FONT_WEIGHT)
    plt.setp(ax.yaxis.get_offset_text(), fontsize=FONT_SIZE_TICK_OFFSET_TEXT, fontweight=FONT_WEIGHT)
    fig.tight_layout()
    ax.set_xlabel(r'# of Rounds $T$', fontsize=FONT_SIZE, fontweight=FONT_WEIGHT)
    ax.set_xticks(x_array)
    xticklabels = [ "{:d}".format(num_round) for num_round in num_round_list ]
    ax.set_xticklabels(xticklabels)
    ax.set_ylabel(r'Average Reward', fontsize=FONT_SIZE, fontweight=FONT_WEIGHT)
    # Save the figure
    fig.savefig('./simulations_data/ht/{:s}/figures/fig_ht_simulations_converged_time_averaged_reward.pdf'.format(date), format='pdf', bbox_extra_artists=(legend,), bbox_inches='tight')
    fig.savefig('./simulations_data/ht/{:s}/figures/fig_ht_simulations_converged_time_averaged_reward.png'.format(date), format='png', bbox_extra_artists=(legend,), bbox_inches='tight')
    
    # For the Chinese version
    
    fig, ax = plt.subplots(figsize=FIG_SIZE_CHINESE, dpi=DPI_CHINESE)
    
    ax.set_ylim(0.15, 0.55)
    
    if CHINESE_FONT_WEIGHT == "bold":
        label_list = [ "$\mathbf{{H_{\mathrm{min}} = 0}}$", "$\mathbf{{H_{\mathrm{min}} = 0.1 T}}$", "$\mathbf{{H_{\mathrm{min}} = T}}$", "$\mathbf{{H_{\mathrm{min}} = T \log T}}$" ]
    else:
        label_list = [ "$H_{\mathrm{min}} = 0$", "$H_{\mathrm{min}} = 0.1 T$", "$H_{\mathrm{min}} = T$", "$H_{\mathrm{min}} = T \log T$" ]

    x_array = np.arange(len(num_round_list))
    width = 0.18 # The width of the bars
    
    for index, converged_time_averaged_compound_reward_array in enumerate(converged_time_averaged_compound_reward_array_list):
        ax.bar(x_array + (index - 1.5) * width, converged_time_averaged_compound_reward_array, width, color=color_list[index], label=label_list[index], edgecolor=BAR_EDGE_COLOR, zorder=3)

    
    if CHINESE_FONT_WEIGHT == "bold":
        legend = ax.legend(shadow=False, ncol=2, prop=fm.FontProperties(size=20.0, style="normal", fname='./font/source_han_serif_sc_bold.otf', math_fontfamily="stix"))
    else:
        legend = ax.legend(shadow=False, ncol=2, prop=fm.FontProperties(size=20.0, style="normal", fname='./font/source_han_serif_sc_regular.otf', math_fontfamily="stix"))
    plt.grid(GRID, linestyle=GRID_LINE_STYLE, zorder=0)
    plt.ticklabel_format(style='sci', axis='y', scilimits=(0, 0), useMathText=True)
    plt.setp(ax.get_xticklabels(), fontsize=FONT_SIZE_TICK_CHINESE, fontweight=CHINESE_FONT_WEIGHT, fontproperties=CHINESE_FP)
    plt.setp(ax.get_yticklabels(), fontsize=FONT_SIZE_TICK_CHINESE, fontweight=CHINESE_FONT_WEIGHT, fontproperties=CHINESE_FP)
    plt.setp(ax.xaxis.get_offset_text(), fontsize=FONT_SIZE_TICK_OFFSET_TEXT_CHINESE, fontweight=CHINESE_FONT_WEIGHT, fontproperties=CHINESE_FP)
    plt.setp(ax.yaxis.get_offset_text(), fontsize=FONT_SIZE_TICK_OFFSET_TEXT_CHINESE, fontweight=CHINESE_FONT_WEIGHT, fontproperties=CHINESE_FP)
    fig.tight_layout()
    ax.set_xlabel(r'总轮数 $T$', fontsize=FONT_SIZE_CHINESE, fontweight=CHINESE_FONT_WEIGHT, fontproperties=CHINESE_FP)
    ax.set_xticks(x_array)
    xticklabels = [ "{:d}".format(num_round) for num_round in num_round_list ]
    ax.set_xticklabels(xticklabels)
    ax.set_ylabel(r'平均奖励', fontsize=FONT_SIZE_CHINESE, fontweight=CHINESE_FONT_WEIGHT, fontproperties=CHINESE_FP)
    # Save the figure
    fig.savefig('./simulations_data/ht/{:s}/chinese_figures/fig_ht_simulations_converged_time_averaged_reward.png'.format(date), format='png', bbox_extra_artists=(legend,), bbox_inches='tight')
    if SAVE_CHINESE_PDF:
        fig.savefig('./simulations_data/ht/{:s}/chinese_figures/fig_ht_simulations_converged_time_averaged_reward.pdf'.format(date), format='pdf', bbox_extra_artists=(legend,), bbox_inches='tight')
    
    
    # Ending for the converged time-averaged compound reward
    ##########################################################################################

    
    # For the converged time-averaged regret
    fig, ax = plt.subplots(figsize=FIG_SIZE, dpi=DPI)
    plt.setp(ax.get_xticklabels(), fontsize=FONT_SIZE_TICK, fontweight=FONT_WEIGHT)
    plt.setp(ax.get_yticklabels(), fontsize=FONT_SIZE_TICK, fontweight=FONT_WEIGHT)
    ax.yaxis.get_offset_text().set_fontsize(20)
    ax.yaxis.get_offset_text().set_fontweight(FONT_WEIGHT)
    ax.xaxis.get_offset_text().set_fontsize(20)
    ax.xaxis.get_offset_text().set_fontweight(FONT_WEIGHT)
    
    label_list = [ "$H_{\mathrm{min}} = 0$", "$H_{\mathrm{min}} = 0.1 T$", "$H_{\mathrm{min}} = T$", "$H_{\mathrm{min}} = T \log T$" ]

    x_array = np.arange(len(num_round_list))
    width = 0.18 # The width of the bars
    
    for index, converged_time_averaged_regret_array in enumerate(converged_time_averaged_regret_array_list):
        ax.bar(x_array + (index - 1.5) * width, converged_time_averaged_regret_array, width, color=color_list[index], label=label_list[index], edgecolor=BAR_EDGE_COLOR, zorder=3)

#     ax.bar_label(rects1, padding=3)
#     ax.bar_label(rects2, padding=3)

    
    legend = ax.legend(shadow=True, prop={ 'size': 32, 'weight': 'normal'})
    plt.grid(GRID, linestyle=GRID_LINE_STYLE, zorder=0)
    fig.tight_layout()
    ax.set_xlabel(r'Time Horizon Length $T$', fontsize=FONT_SIZE, fontweight=FONT_WEIGHT)
    ax.set_xticks(x_array)
    xticklabels = [ "{:d}".format(num_round) for num_round in num_round_list ]
#     plt.ticklabel_format(style='sci', axis='y', scilimits=(0, 0))
    ax.set_xticklabels(xticklabels)
    ax.set_ylabel(r'Converged Average Regret', fontsize=FONT_SIZE, fontweight=FONT_WEIGHT)
    # Save the figure
    fig.savefig('./simulations_data/ht/{:s}/figures/fig_ht_simulations_converged_time_averaged_regret.pdf'.format(date), format='pdf', bbox_extra_artists=(legend,), bbox_inches='tight')
    fig.savefig('./simulations_data/ht/{:s}/figures/fig_ht_simulations_converged_time_averaged_regret.png'.format(date), format='png', bbox_extra_artists=(legend,), bbox_inches='tight')


matplotlib.__version__: 3.5.1
Default backend: module://matplotlib_inline.backend_inline
Backend is now module://mplcairo.base


In [4]:
plot_ht_simulations()
